In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import chart_studio.plotly as py
import plotly.graph_objs as go
from sklearn import svm
from sklearn.metrics import roc_auc_score, log_loss
from sklearn.model_selection import StratifiedKFold
from methods import *

In [2]:
%load_ext autoreload
%autoreload 2

clean: 552
polygon: 276
kelvin: 63
gotham: 51
lomo: 60
nashville: 55
toaster: 47

In [3]:
# def read_features(p_path, p_arch_to_select=None, p_trigger_type=None):
def read_features(p_path):
    # features for Round 2
    column_names_for_features = [
        'square_mean_diff', 'square_std_diff',
        'gotham_mean_diff', 'gotham_std_diff',
        'kelvin_mean_diff', 'kelvin_std_diff',
        'lomo_mean_diff', 'lomo_std_diff',
        'nashville_mean_diff', 'nashville_std_diff',
        'toaster_mean_diff', 'toaster_std_diff'
    ]
    report = pd.read_csv(p_path)
#     if p_arch_to_select is not None:
#         report = report[report['model_architecture'] == p_arch_to_select]
#     if p_trigger_type is not None:
#         indexes = []
#         for i in range(len(report)):
#             tta = report['trigger_type_aux'].iloc[i].lower()
# #             if (p_trigger_type in tta):
#             if ('none' in tta):
#                 indexes.append(i)
#         report = report.iloc[indexes, :]
    features = report[column_names_for_features].values
    labels = np.array([int(report['model_label'].iloc[i] == 'backdoor') for i in range(len(report))])
    return abs(features), labels

def evaluate_classifier(train_x, train_y, test_x, test_y):
    clf = svm.SVC(kernel='linear', probability=True)
    clf.fit(train_x, train_y)
    y_score = clf.predict(test_x)
    y_pred = clf.predict_proba(test_x)
    roc_auc = roc_auc_score(y_true=test_y, y_score=y_score)
    cross_entropy = log_loss(y_true=test_y, y_pred=y_pred)
    return roc_auc, cross_entropy

# stratified 5-fold validation for square data (apply Round-1 approach)

In [16]:
trigger_data_to_use = 'polygon'
# trigger_data_to_use = 'gotham'
# trigger_data_to_use = 'kelvin'
# trigger_data_to_use = 'lomo'
# trigger_data_to_use = 'nashville'
# trigger_data_to_use = 'toaster'

dict_trigger_indexes = { 'polygon': 0, 'gotham': 2, 'kelvin': 4, 'lomo': 6, 'nashville': 8, 'toaster': 10 }
start_index = dict_trigger_indexes[trigger_data_to_use]

path_csv = r'confusion-reports\ics_svm\round2-train-dataset\round2-train-dataset_square20-gotham-kelvin-lomo-nashville-toaster.csv'

X, y = read_features(p_path=path_csv) # clean data is automatically added
skf = StratifiedKFold(n_splits=5, shuffle=True)

scores_roc, scores_xent = [], []

for index, (train_index, test_index) in enumerate(skf.split(X, y)):
    X_train, y_train = X[train_index, start_index:start_index+2], y[train_index] # square_diffs have indexes 0,1 !!!
    X_test, y_test = X[test_index, start_index:start_index+2], y[test_index] # square_diffs have indexes 0,1 !!!
    
    roc, xent = evaluate_classifier(X_train, y_train, X_test, y_test)
    
    scores_roc.append(roc)
    scores_xent.append(xent)

print(f'5-fold CV ({trigger_data_to_use})')
print(f'avg roc ={sum(scores_roc) / len(scores_roc)}')
print(f'std roc ={np.std(scores_roc)}')
print()
print(f'avg xent={sum(scores_xent) / len(scores_xent)}')
print(f'std roc ={np.std(scores_xent)}')

5-fold CV (polygon)
avg roc =0.6982637182637184
std roc =0.028185608275125677

avg xent=0.5640477865282053
std roc =0.02175045249869255


# stratified 5-fold validation for all data (square + 5 filters)

In [4]:
path_csv = r'confusion-reports\ics_svm\round2-train-dataset\round2-train-dataset_square20-gotham-kelvin-lomo-nashville-toaster.csv'
# dict_trigger_indexes = { 'polygon': 0, 'gotham': 2, 'kelvin': 4, 'lomo': 6, 'nashville': 8, 'toaster': 10 }
X, y = read_features(p_path=path_csv) # clean data is automatically added
skf = StratifiedKFold(n_splits=5, shuffle=True)
scores_roc, scores_xent = [], []
for index, (train_index, test_index) in enumerate(skf.split(X, y)):
    X_train, y_train = X[train_index, :], y[train_index]
    X_test, y_test = X[test_index, :], y[test_index]
#     print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)
    roc, xent = evaluate_classifier(X_train, y_train, X_test, y_test)
#     print(f'split #{index}: roc={roc:.4f}, xent={xent:.4f}')
    scores_roc.append(roc)
    scores_xent.append(xent)
print(f'avg roc ={sum(scores_roc) / len(scores_roc)}')
print(f'std roc ={np.std(scores_roc)}')
print()
print(f'avg xent={sum(scores_xent) / len(scores_xent)}')
print(f'std roc ={np.std(scores_xent)}')

avg roc =0.8903931203931205
std roc =0.01988645636664563

avg xent=0.2901835012449866
std roc =0.038958773935452023


# Plot (mean, std)  of clean data vs (mean, std) of backdoored data (polygon or instagram filter)

In [ ]:
path_csv = r'confusion-reports\ics_svm\round2-train-dataset\round2-train-dataset_square20-gotham-kelvin-lomo-nashville-toaster.csv'
dict_trigger_indexes = { 'polygon': 0, 'gotham': 2, 'kelvin': 4, 'lomo': 6, 'nashville': 8, 'toaster': 10 }
X_train, y_train = read_features(p_path=path_csv) # clean data is automatically added

for trigger_type, idx in dict_trigger_indexes.items():
    print(trigger_type, idx)
    x_axis = X_train[:, idx]
    y_axis = X_train[:, idx+1]
    
#     list_labels = [['clean', 'backdoored'][y] for y in y_train]
#     list_colors = [['#1f77b4', 'orange'][y] for y in y_train]
    
#     plt.figure(figsize=(10, 6)).patch.set_color('white')
#     plt.scatter(x_axis, y_axis, c=list_colors, s=20, cmap=plt.cm.Paired)

# #     count_backdoored = 0
# #     count_clean = 0
# #     for label_int, label_name, color in [(0, 'clean', '#1f77b4'), (1, 'backdoored', 'orange')]:
# #         mask = (y_train == label_int)
# #         if label_int == 0:
# #             count_clean = mask.sum()
# #         else:
# #             count_backdoored = mask.sum()
# #         plt.scatter(x_axis[mask], y_axis[mask], c=color, zorder=10, s=20, cmap=plt.cm.Paired, label=label_name)

# #     plt.xlim([0, 20])
# #     plt.ylim([0, 6])
# #     plt.axvline(x=3, ymin=y_axis.min(), ymax=y_axis.max(), color='k')
#     plt.xlabel('mean diff')
#     plt.ylabel('std diff')
#     plt.title(f'({count_clean} clean models) VS ({count_backdoored} {trigger_type}-trigger models)')
#     plt.grid()
# #     plt.legend()
#     plt.show()

# Plot

## show (size vs ROC) plots for training and holdout for individual architectures

In [ ]:
list_square_sizes = [5, 10, 15, 20, 25, 30, 35, 40, 45, 50]
list_archs = ['densenet121', 'resnet50', 'inceptionv3']

for dataset in ['round1-dataset-train', 'round1-holdout-dataset']:
    roc_auc_scores = {arch:[] for arch in list_archs}
    xent_scores = {arch:[] for arch in list_archs}
    for square_size in list_square_sizes:
        for arch in list_archs:
            path = f'confusion-reports/ics_svm/{dataset}/{dataset}_custom-square-size-{square_size}_backd-original-color_clean-black-color.csv'
            X, y = read_features(path, arch)
            roc, xent = evaluate_classifier(X, y, X, y)
            roc_auc_scores[arch].append(roc)
            xent_scores[arch].append(xent)
    
    print('#############', dataset, '#############')
    # PLOT ROC
    plt.figure(figsize=(10, 6)).patch.set_color('white')
    for arch in roc_auc_scores.keys():
        plt.plot(list_square_sizes, roc_auc_scores[arch], 'o-', label=arch)
    plt.grid()
    plt.legend()
    plt.xlabel('square trigger size')
    plt.ylabel('ROC AUC for SVM classifier')
    plt.xticks(list_square_sizes)
    plt.title(f'trigger size vs ROC for {dataset} (architecture-wise)')
    plt.show()
    
    # PLOT XENT
    plt.figure(figsize=(10, 6)).patch.set_color('white')
    for arch in roc_auc_scores.keys():
        plt.plot(list_square_sizes, xent_scores[arch], 'o-', label=arch)
    plt.grid()
    plt.legend()
    plt.xlabel('square trigger size')
    plt.ylabel('Cross-Entropy for SVM classifier')
    plt.xticks(list_square_sizes)
    plt.title(f'trigger size vs Cross-Entropy for {dataset} (architecture-wise)')
    plt.show()

## show (size vs ROC) plots for training and holdout using all architectures as a whole

In [ ]:
list_square_sizes = [5, 10, 15, 20, 25, 30, 35, 40, 45, 50]
list_archs = ['densenet121', 'resnet50', 'inceptionv3']

for dataset in ['round1-dataset-train', 'round1-holdout-dataset']:
    roc_auc_scores = []
    xent_scores = []
    for square_size in list_square_sizes:
        path = f'confusion-reports/ics_svm/{dataset}/{dataset}_custom-square-size-{square_size}_backd-original-color_clean-black-color.csv'
        X, y = read_features(path)
        
        roc, xent = evaluate_classifier(X, y, X, y)
        roc_auc_scores.append(roc)
        xent_scores.append(xent)
        
    print('#############', dataset, '#############')
    plt.figure(figsize=(10, 6)).patch.set_color('white')
    plt.plot(list_square_sizes, roc_auc_scores, 'o-', label='all archs')
    plt.grid()
    plt.legend()
    plt.xlabel('square trigger size')
    plt.ylabel('ROC AUC for SVM classifier')
    plt.xticks(list_square_sizes)
    plt.title(f'trigger size vs ROC for {dataset} (all architectures combined)')
    plt.show()
    
    plt.figure(figsize=(10, 6)).patch.set_color('white')
    plt.plot(list_square_sizes, xent_scores, 'o-', label='all archs')
    plt.grid()
    plt.legend()
    plt.xlabel('square trigger size')
    plt.ylabel('Cross-Entropy for SVM classifier')
    plt.xticks(list_square_sizes)
    plt.title(f'trigger size vs Cross-Entropy for {dataset} (all architectures combined)')
    plt.show()

## train an SVM classifier on training with trigger_size=S and test it on holdout data with trigger_size=S

In [ ]:
report = pd.DataFrame(columns=['train_size', 'holdout_size', 'roc_train_holdout', 'xent_train_holdout'])
n_report = 0
for size_train in list_square_sizes:
    path_train = f'confusion-reports/ics_svm/round1-dataset-train/round1-dataset-train_custom-square-size-{size_train}_backd-original-color_clean-black-color.csv'
    X_train, y_train = read_features(path_train)
    
    for size_holdout in list_square_sizes:
        path_holdout = f'confusion-reports/ics_svm/round1-holdout-dataset/round1-holdout-dataset_custom-square-size-{size_holdout}_backd-original-color_clean-black-color.csv'
        X_holdout, y_holdout = read_features(path_holdout)
        roc_train_holdout, xent_train_holdout = evaluate_classifier(X_train, y_train, X_holdout, y_holdout)
        
        report.loc[n_report] = [size_train, size_holdout, roc_train_holdout, xent_train_holdout]
        n_report += 1
        print(f'size_train={size_train}, size_holdout={size_holdout}: roc={roc_train_holdout}, xent={xent_train_holdout}')
report.to_csv('confusion-reports/ics_svm/trained-on-train-tested-on-holdout.csv', index=False)

In [ ]:
# def merge_datasets():
#     """I need to add the columns toaster_mean_diff, toaster_std_diff, toaster_mean, toaster_std from df2 to df1 and reorder the columns accordingly"""
#     df1 = pd.read_csv(r'confusion-reports\ics_svm\round2-train-dataset\round2-train-dataset_square20-gotham-kelvin-lomo-nashville.csv')
#     df2 = pd.read_csv(r'confusion-reports\ics_svm\round2-train-dataset\round2-train-dataset_toaster.csv')
#     for col in ['toaster_mean_diff', 'toaster_std_diff', 'toaster_mean', 'toaster_std']:
#         df1[col] = df2[col]
    
#     series_data = []
#     for i in range(len(df1)):
#         trigger_type = df1.iloc[i]['trigger_type']
#         trigger_type_option = df1.iloc[i]['trigger_type_option']
#         if trigger_type == 'instagram':
#             series_data.append(trigger_type_option.replace('XForm', '').replace('Filter', ''))
#         else:
#             if trigger_type == 'None':
#                 series_data.append(trigger_type)
#             else:
#                 series_data.append(f'{trigger_type}-{trigger_type_option}')
#     df1['trigger_type_aux'] = series_data
    
#     new_order_for_columns = [
#         'model_name', 'model_architecture', 'model_label',
#         'trigger_type_aux', # 'trigger_type', 'trigger_type_option',

#         'square_mean_diff', 'square_std_diff',
#         'gotham_mean_diff', 'gotham_std_diff',
#         'kelvin_mean_diff', 'kelvin_std_diff',
#         'lomo_mean_diff', 'lomo_std_diff',
#         'nashville_mean_diff', 'nashville_std_diff',
#         'toaster_mean_diff', 'toaster_std_diff',

#         'clean_mean', 'clean_std',
#         'square_mean', 'square_std',
#         'gotham_mean', 'gotham_std',
#         'kelvin_mean', 'kelvin_std',
#         'lomo_mean', 'lomo_std',
#         'nashville_mean', 'nashville_std',
#         'toaster_mean', 'toaster_std',

#         'trigger_color', 'num_classes',
#     ]
#     df1 = df1[new_order_for_columns]
#     df1.to_csv(r'confusion-reports\ics_svm\round2-train-dataset\round2-train-dataset_square20-gotham-kelvin-lomo-nashville-toaster.csv', index=False)
#     print('done')
# # merge_datasets()